**Phụ lục D – Autodiff**

_Noteboook này chứa các khai triển ví dụ của các kỹ thuật autodiff khác nhau để giải thích cách mà chúng hoạt động._

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/mlbvn/handson-ml2-vn/blob/main/extra_autodiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/mlbvn/handson-ml2-vn/blob/main/extra_autodiff.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# Cài đặt

# Giới thiệu

Giả sử ta cần tính gradient của hàm  $f(x,y)=x^2y + y + 2$ theo tham số x và y:

In [1]:
def f(x,y):
    return x*x*y + y + 2

Một phương pháp để giải bài toán một cách phân tích:

$\dfrac{\partial f}{\partial x} = 2xy$

$\dfrac{\partial f}{\partial y} = x^2 + 1$

In [2]:
def df(x,y):
    return 2*x*y, x*x + 1

Ví dụ $\dfrac{\partial f}{\partial x}(3,4) = 24$ và $\dfrac{\partial f}{\partial y}(3,4) = 10$.

In [3]:
df(3, 4)

(24, 10)

Hoàn hảo! Ta cũng có thể tìm phương trình cho đạo hàm bậc hai (còn được gọi là Hessian):

$\dfrac{\partial^2 f}{\partial x \partial x} = \dfrac{\partial (2xy)}{\partial x} = 2y$

$\dfrac{\partial^2 f}{\partial x \partial y} = \dfrac{\partial (2xy)}{\partial y} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial x} = \dfrac{\partial (x^2 + 1)}{\partial x} = 2x$

$\dfrac{\partial^2 f}{\partial y \partial y} = \dfrac{\partial (x^2 + 1)}{\partial y} = 0$

Tại x=3 và y=4, các Hessian lần lượt là 8, 6, 6, 0. Hãy sử dụng các phương trình trên để tính chúng:

In [4]:
def d2f(x, y):
    return [2*y, 2*x], [2*x, 0]

In [5]:
d2f(3, 4)

([8, 6], [6, 0])

Tuyệt, nhưng ta cần phải làm một số toán. Không quá khó trong trường hợp này nhưng cho mạng nơ-ron sâu, trên thực tế không thể tính đạo hàm bằng cách này. Vì vậy hãy xem một vài cách để tự động hoá tính toán!

# Tính vi phân số học

Ở đây, ta tính xấp xỉ gradient sử dụng phương trình: $\dfrac{\partial f}{\partial x} = \displaystyle{\lim_{\epsilon \to 0}}\dfrac{f(x+\epsilon, y) - f(x, y)}{\epsilon}$ (và có một định nghĩa tương tự cho  $\dfrac{\partial f}{\partial y}$)

In [6]:
def gradients(func, vars_list, eps=0.0001):
    partial_derivatives = []
    base_func_eval = func(*vars_list)
    for idx in range(len(vars_list)):
        tweaked_vars = vars_list[:]
        tweaked_vars[idx] += eps
        tweaked_func_eval = func(*tweaked_vars)
        derivative = (tweaked_func_eval - base_func_eval) / eps
        partial_derivatives.append(derivative)
    return partial_derivatives

In [7]:
def df(x, y):
    return gradients(f, [x, y])

In [8]:
df(3, 4)

[24.000400000048216, 10.000000000047748]

Nó hoạt động tốt!

Tin tốt là tính Hessian khá dễ. Đầu tiên hãy tạo các hàm tính đạo hàm bậc một (còn gọi là Jacobian):

In [9]:
def dfdx(x, y):
    return gradients(f, [x,y])[0]

def dfdy(x, y):
    return gradients(f, [x,y])[1]

dfdx(3., 4.), dfdy(3., 4.)

(24.000400000048216, 10.000000000047748)

Bây giờ ta có thể đơn giản áp dụng hàm  `gradients()` vào các hàm này:

In [10]:
def d2f(x, y):
    return [gradients(dfdx, [x, y]), gradients(dfdy, [x, y])]

In [11]:
d2f(3, 4)

[[7.999999951380232, 6.000099261882497],
 [6.000099261882497, -1.4210854715202004e-06]]

Vậy là mọi thứ hoạt động tốt, nhưng kết quả được xấp xỉ, và tính toán gradient của một hàm theo $n$ biến cần gọi hàm đó $n$ lần. Trong các mạng nơ-ron sâu, thường có hàng ngàn tham số để tinh chỉnh sử dụng phương pháp hạ gradient (điều này cần tính toán gradient của hàm mất mát theo mỗi tham số), vì vậy phương pháp này sẽ rất chậm.

## Triển khai một Đồ thị Tính toán Giả lập

Thay vì phương pháp số học này, hãy triển khai một vài phương pháp autodiff biểu trưng. Để làm điều này, ta cần định nghĩa các lớp để miêu tả hằng số, biến số và các toán tử. 

In [12]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
    def evaluate(self):
        return self.value
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() + self.b.evaluate()
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        return self.a.evaluate() * self.b.evaluate()
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

Tốt, bây giờ ta có thể xây dựng một đồ thị tính toán để miêu tả hàm $f$:

In [13]:
x = Var("x")
y = Var("y")
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

Và ta có thể chạy đồ thị này để tính $f$ tại bất cứ điểm nào, ví dụ $f(3, 4)$.

In [14]:
x.value = 3
y.value = 4
f.evaluate()

42

Hoàn hảo, ta đã tìm thấy kết quả cuối cùng.

## Tính toán gradient

Các phương pháp autodiff ta sẽ trình bày dưới đây đều dựa trên *quy tắc dây chuyền*.

Giả sử ta có hai hàm $u$ và $v$, và ta áp dụng chúng tuần tự cho một số đầu vào $x$, và ta thu được kết quả $z$. Vì thế ta có  $z = v(u(x))$ mà ta có thể viết lại thành  $z = v(s)$ và $s = u(x)$. Bây giờ ta có thể áp dụng quy tắc dây chuyền để thu được đạo hàm riêng của đầu ra $z$ theo đầu vào $x$:

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s}{\partial x} \cdot \dfrac{\partial z}{\partial s}$

Bây giờ nếu $z$ là đầu ra của chuỗi các hàm có các đầu ra trung gian $s_1, s_2, ..., s_n$, quy tắc dây chuyền vẫn được áp dụng:

$ \dfrac{\partial z}{\partial x} = \dfrac{\partial s_1}{\partial x} \cdot \dfrac{\partial s_2}{\partial s_1} \cdot \dfrac{\partial s_3}{\partial s_2} \cdot \dots \cdot \dfrac{\partial s_{n-1}}{\partial s_{n-2}} \cdot \dfrac{\partial s_n}{\partial s_{n-1}} \cdot \dfrac{\partial z}{\partial s_n}$

Trong autodiff thuận, thuật toán tính các số hạng này theo "chiều thuận" (nghĩa là cùng thứ tự với tính toán cần để tính đầu ra $z$), từ trái qua phải: đầu tiên $\dfrac{\partial s_1}{\partial x}$, tiếp theo  $\dfrac{\partial s_2}{\partial s_1}$, v.v. Trong autodiff ngược, thuật toán tính các số hạng này theo "chiều ngược", từ phải qua trái: đầu tiên $\dfrac{\partial z}{\partial s_n}$, tiếp đó $\dfrac{\partial s_n}{\partial s_{n-1}}$, v.v.

Ví dụ, giả sử ta muốn tính đạo hàm của hàm  $z(x)=\sin(x^2)$ tại x=3, sử dụng autodiff thuận. Thuật toán đầu tiên sẽ tính đạo hàm riêng $\dfrac{\partial s_1}{\partial x}=\dfrac{\partial x^2}{\partial x}=2x=6$. Tiếp theo, nó sẽ tính $\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1}= 6 \cdot \dfrac{\partial \sin(s_1)}{\partial s_1}=6 \cdot \cos(s_1)=6 \cdot \cos(3^2)\approx-5.46$.

Hãy kiểm tra kết quả này sử dụng hàm `gradients()` được định nghĩa ở trên:

In [15]:
from math import sin

def z(x):
    return sin(x**2)

gradients(z, [3])

[-5.46761419430053]

Trông có vẻ ổn. Bây giờ hãy làm điều tương tự bằng cách sử dụng autodiff ngược. Lần này thuật toán sẽ bắt đầu từ bên phải vì thế nó sẽ tính $\dfrac{\partial z}{\partial s_1} = \dfrac{\partial \sin(s_1)}{\partial s_1}=\cos(s_1)=\cos(3^2)\approx -0.91$. Tiếp theo nó sẽ tính $\dfrac{\partial z}{\partial x}=\dfrac{\partial s_1}{\partial x}\cdot\dfrac{\partial z}{\partial s_1} \approx \dfrac{\partial s_1}{\partial x} \cdot -0.91 = \dfrac{\partial x^2}{\partial x} \cdot -0.91=2x \cdot -0.91 = 6\cdot-0.91=-5.46$.

Tất nhiên cả hai phương pháp đều cho kết quả như nhau (ngoại trừ lỗi làm tròn), và với một đầu vào và đầu ra duy nhất, chúng cần cùng một số lượng phép tính. Nhưng khi có một vài đầu vào và đầu ra, chúng có thể có hoạt động rất khác nhau. Thật vậy, nếu có nhiều đầu vào, số hạng ở ngoài cùng bên phải sẽ cần được tính đạo hàm riêng theo mỗi đầu vào, do đó, nên tính các số hạng ngoài cùng bên phải trước. Điều này tương đương với việc sử dụng autodiff ngược. 

Bằng cách này, các số hạng ngoài cùng bên phải có thể được tính chỉ một lần và được sử dụng để tính tất cả các đạo hàm riêng. Ngược lại, nếu có nhiều đầu ra, autodiff thuận thường được ưu tiên hơn vì các số hạng ngoài cùng bên trái có thể được tính toán chỉ một lần để tính các đạo hàm riêng của các đầu ra khác nhau. Trong Học Sâu, thường có hàng ngàn tham số mô hình, nghĩa là có rất nhiều đầu vào nhưng ít đầu ra. Trên thực tế, nhìn chung chỉ có một đầu ra trong suốt quá trình huấn luyện: giá trị mất mát. Đây là lý do vì sao autodiff ngược được sử dụng trong TensorFlow và tất cả các thư viện Học Sâu chính.

Có một số phức tạp bổ sung trong autodiff ngược: giá trị của $s_i$ thường cần để tính $\dfrac{\partial s_{i+1}}{\partial s_i}$, và tính $s_i$ cần tính $s_(i-1)$ trước, nghĩa là cần tính  $s_{i-2}$, v.v. Vì vậy về cơ bản, một vòng tính toán thuận xuyên suốt mạng cần thiết để tính  $s_1$, $s_2$, $s_3$, $\dots$, $s_{n-1}$ và $s_n$, và tiếp theo thuật toán có thể tính đạo hàm riêng từ phải qua trái. Lưu tất cả các giá trị trung gian $s_i$ trong RAM thường là một vấn đề, đặc biệt khi xử lý ảnh, và khi sử dụng GPU thường có dung lượng RAM hạn chế: để hạn chế vấn đề này, ta có thể giảm số lượng tầng trong mạng nơ-ron, hoặc cài đặt TensorFlow để hoán đổi các giá trị này từ RAM GPU sang RAM CPU. Một cách khác là chỉ lưu một số giá trị trung gian ở bước lẻ $s_1$, $s_3$, $s_5$, $\dots$, $s_{n-4}$, $s_{n-2}$ và $s_n$. Điều này nghĩa là khi thuật toán tính đạo hàm riêng, nếu một giá trị trung gian $s_i$ bị thiếu, nó sẽ tính lại dựa trên giá trị trung gian trước đó $s_(i-1)$. Việc này đánh đổi CPU với RAM (nếu bạn muốn tìm hiểu thêm, xem [bài báo này](https://pdfs.semanticscholar.org/f61e/9fd5a4878e1493f7a6b03774a61c17b7e9a4.pdf)).

### Autodiff thuận

In [16]:
Const.gradient = lambda self, var: Const(0)
Var.gradient = lambda self, var: Const(1) if self is var else Const(0)
Add.gradient = lambda self, var: Add(self.a.gradient(var), self.b.gradient(var))
Mul.gradient = lambda self, var: Add(Mul(self.a, self.b.gradient(var)), Mul(self.a.gradient(var), self.b))

x = Var(name="x", init_value=3.)
y = Var(name="y", init_value=4.)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

dfdx = f.gradient(x)  # 2xy
dfdy = f.gradient(y)  # x² + 1

In [17]:
dfdx.evaluate(), dfdy.evaluate()

(24.0, 10.0)

Vì đầu ra của phương thức `gradient()` thì hoàn toàn biểu trưng, ta không bị giới hạn ở việc chỉ tính đạo hàm bậc một, ta cũng có thể tính đạo hàm bậc hai, v.v.:

In [18]:
d2fdxdx = dfdx.gradient(x) # 2y
d2fdxdy = dfdx.gradient(y) # 2x
d2fdydx = dfdy.gradient(x) # 2x
d2fdydy = dfdy.gradient(y) # 0

In [19]:
[[d2fdxdx.evaluate(), d2fdxdy.evaluate()],
 [d2fdydx.evaluate(), d2fdydy.evaluate()]]

[[8.0, 6.0], [6.0, 0.0]]

Lưu ý rằng kết quả bây giờ là chính xác, không phải là xấp xỉ (tất nhiên là đến giới hạn độ chính xác số động của máy tính).

### Autodiff thuận sử dụng các số đối ngẫu

Một cách hay để áp dụng autodiff thuận là sử dụng [các số đối ngẫu](https://en.wikipedia.org/wiki/Dual_number). Nói một cách ngắn gọn, một số đối ngẫu $z$ có dạng  $z = a + b\epsilon$, trong đó $a$ và $b$ là các số thực, và  $\epsilon$ là một số cực nhỏ, dương nhưng nhỏ hơn tất cả các số thực, và sao cho  $\epsilon^2=0$.
Ta có thể chứng minh  $f(x + \epsilon) = f(x) + \dfrac{\partial f}{\partial x}\epsilon$, vì thế đơn giản bằng cách tính $f(x + \epsilon)$ ta có thể thu được cả giá trị của $f(x)$ và đạo hàm riên của $f$ theo $x$.

Các số đối ngẫu có quy tắc số học riêng của chúng, thường khá là tự nhiên. Ví dụ:

**Phép cộng**

$(a_1 + b_1\epsilon) + (a_2 + b_2\epsilon) = (a_1 + a_2) + (b_1 + b_2)\epsilon$

**Phép trừ**

$(a_1 + b_1\epsilon) - (a_2 + b_2\epsilon) = (a_1 - a_2) + (b_1 - b_2)\epsilon$

**Phép nhân**

$(a_1 + b_1\epsilon) \times (a_2 + b_2\epsilon) = (a_1 a_2) + (a_1 b_2 + a_2 b_1)\epsilon + b_1 b_2\epsilon^2 = (a_1 a_2) + (a_1b_2 + a_2b_1)\epsilon$

**Phép chia**

$\dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} = \dfrac{a_1 + b_1\epsilon}{a_2 + b_2\epsilon} \cdot \dfrac{a_2 - b_2\epsilon}{a_2 - b_2\epsilon} = \dfrac{a_1 a_2 + (b_1 a_2 - a_1 b_2)\epsilon - b_1 b_2\epsilon^2}{{a_2}^2 + (a_2 b_2 - a_2 b_2)\epsilon - {b_2}^2\epsilon} = \dfrac{a_1}{a_2} + \dfrac{a_1 b_2 - b_1 a_2}{{a_2}^2}\epsilon$

**Luỹ thừa**

$(a + b\epsilon)^n = a^n + (n a^{n-1}b)\epsilon$

v.v.

Hãy tạo một lớp để mô tả các số đối ngẫu, và triển khai một số toán tử (cộng và nhân). Bạn có thể thử thêm nhiều hơn nếu bạn muốn.

In [20]:
class DualNumber(object):
    def __init__(self, value=0.0, eps=0.0):
        self.value = value
        self.eps = eps
    def __add__(self, b):
        return DualNumber(self.value + self.to_dual(b).value,
                          self.eps + self.to_dual(b).eps)
    def __radd__(self, a):
        return self.to_dual(a).__add__(self)
    def __mul__(self, b):
        return DualNumber(self.value * self.to_dual(b).value,
                          self.eps * self.to_dual(b).value + self.value * self.to_dual(b).eps)
    def __rmul__(self, a):
        return self.to_dual(a).__mul__(self)
    def __str__(self):
        if self.eps:
            return "{:.1f} + {:.1f}ε".format(self.value, self.eps)
        else:
            return "{:.1f}".format(self.value)
    def __repr__(self):
        return str(self)
    @classmethod
    def to_dual(cls, n):
        if hasattr(n, "value"):
            return n
        else:
            return cls(n)

$3 + (3 + 4 \epsilon) = 6 + 4\epsilon$

In [21]:
3 + DualNumber(3, 4)

6.0 + 4.0ε

$(3 + 4ε)\times(5 + 7ε)$ = $3 \times 5 + 3 \times 7ε + 4ε \times 5 + 4ε \times 7ε$ = $15 + 21ε + 20ε + 28ε^2$ = $15 + 41ε + 28 \times 0$ = $15 + 41ε$

In [22]:
DualNumber(3, 4) * DualNumber(5, 7)

15.0 + 41.0ε

Bây giờ hãy xem liệu các số đối ngẫu hoạt động với khung tính toán giả lập không:

In [23]:
x.value = DualNumber(3.0)
y.value = DualNumber(4.0)

f.evaluate()

42.0

Vâng, chắc chắn hoạt động. Bây giờ hãy sử dụng cái này để tính các đạo hàm riêng của $f$ theo $x$ và $y$ tại x=3 và y=4:

In [24]:
x.value = DualNumber(3.0, 1.0)  # 3 + ε
y.value = DualNumber(4.0)       # 4

dfdx = f.evaluate().eps

x.value = DualNumber(3.0)       # 3
y.value = DualNumber(4.0, 1.0)  # 4 + ε

dfdy = f.evaluate().eps

In [25]:
dfdx

24.0

In [26]:
dfdy

10.0

Tuyệt! Tuy nhiên, trong triển khai này ta bị giới hạn với các đạo hàm bậc một. Bây giờ hãy xem autodiff ngược.

### Autodiff ngược

Hãy viết lại khung giả lập để thêm autodiff ngược:

In [27]:
class Const(object):
    def __init__(self, value):
        self.value = value
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        pass
    def __str__(self):
        return str(self.value)

class Var(object):
    def __init__(self, name, init_value=0):
        self.value = init_value
        self.name = name
        self.gradient = 0
    def evaluate(self):
        return self.value
    def backpropagate(self, gradient):
        self.gradient += gradient
    def __str__(self):
        return self.name

class BinaryOperator(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b

class Add(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() + self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient)
        self.b.backpropagate(gradient)
    def __str__(self):
        return "{} + {}".format(self.a, self.b)

class Mul(BinaryOperator):
    def evaluate(self):
        self.value = self.a.evaluate() * self.b.evaluate()
        return self.value
    def backpropagate(self, gradient):
        self.a.backpropagate(gradient * self.b.value)
        self.b.backpropagate(gradient * self.a.value)
    def __str__(self):
        return "({}) * ({})".format(self.a, self.b)

In [28]:
x = Var("x", init_value=3)
y = Var("y", init_value=4)
f = Add(Mul(Mul(x, x), y), Add(y, Const(2))) # f(x,y) = x²y + y + 2

result = f.evaluate()
f.backpropagate(1.0)

In [29]:
print(f)

((x) * (x)) * (y) + y + 2


In [30]:
result

42

In [31]:
x.gradient

24.0

In [32]:
y.gradient

10.0

Một lần nữa, trong triển khai này các đầu ra chỉ là số, không phải là biểu thức biểu trưng, vì vậy ta chỉ tính đến đạo hàm bậc một. Tuy nhiên, ta cũng có các phương thức `backpropagate()` trả về biểu thức biểu trưng thay vì giá trị (nghĩa là trả về `Add(2,3)` thay vì 5). Điều này sẽ giúp tính toán gradient bậc hai (và hơn nữa). Đây là điều mà TensorFlow làm, cũng như là tất cả các thư viện chính triển khai autodiff.

### Autodiff ngược sử dụng TensorFlow

In [33]:
import tensorflow as tf

In [34]:
x = tf.Variable(3.)
y = tf.Variable(4.)

with tf.GradientTape() as tape:
    f = x*x*y + y + 2

jacobians = tape.gradient(f, [x, y])
jacobians

[<tf.Tensor: shape=(), dtype=float32, numpy=24.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

Vì mọi thứ đều biểu trưng, ta có thể tính đạo hàm bậc hai, và hơn thế nữa:

In [35]:
x = tf.Variable(3.)
y = tf.Variable(4.)

with tf.GradientTape(persistent=True) as tape:
    f = x*x*y + y + 2
    df_dx, df_dy = tape.gradient(f, [x, y])

d2f_d2x, d2f_dydx = tape.gradient(df_dx, [x, y])
d2f_dxdy, d2f_d2y = tape.gradient(df_dy, [x, y])
del tape

hessians = [[d2f_d2x, d2f_dydx], [d2f_dxdy, d2f_d2y]]
hessians

[[<tf.Tensor: shape=(), dtype=float32, numpy=8.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=6.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=6.0>, None]]

Lưu ý rằng khi tính đạo hàm của một tensor theo một biến mà nó không phụ thuộc, thay vì trả về 0.0, hàm  `gradient()` trả về `None`.

Và đến đây là hết rồi tất cả các bạn! Hy vọng mọi người thích notebook này.